# Model Training and Testing
Only the video data in its first 15 seconds of one file indexed 3396 is used to illustrate the model training and testing process with time efficiency.
### Pre-setting
- Import relevant modules
- Define preconfigured global variables

In [1]:
import json
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from pre_processing.utils import build_bandpass_filter
from models.cnn3d import CNN

torch.manual_seed(1) 
FPS = 61
WINDOW_LEN = FPS * 10 # frame number for 10-second video clips
time_len = 15 * FPS

### Load and Pre-process Samples of Dataset
- Load the three pre-processed HCI data files:  
    seq_dir: directory of multiple json files, each json file contains a sequence of detected bounding box of one video  
    ts_file: path of a file, the file contains the groundtruth timestamps of BVP peaks for videos in HCI database  
    instan_hr_file: path of a file, the file contains the computed groundtruth of instantaneous heart rate for videos in HCI database  
- Pre-process the data of bounding-box sequences:   
resize -> bandpath filter -> syncronization -> groundtruth computation -> apply sliding window strategy to construct samples with the same time length
- Obtain samples: [(ROI sequence in RGB channels, mean value of HR groundtruth), ...]  
    the shape of ROI sequence in RGB channels = (n=610, c=3, w=36, h=36)

In [2]:
def preprocess_HCI_dataset_for_3dcnn_model(seq_dir, ts_file, instant_hr_file):
    with open(ts_file, 'r') as file:
        timestamps = json.load(file)
    with open(instant_hr_file, 'r') as file:
        instant_hr = json.load(file)

    sequences = {}
    bd_files = os.listdir(seq_dir)
    if '.DS_Store' in bd_files:
        bd_files.remove('.DS_Store')
    for bd_file in bd_files:
        with open(seq_dir + bd_file, 'r') as file:
            sequences[bd_file[:-5]] = json.load(file)

    print("All Files:", sequences.keys())
    all_data = []
    for file_idx in sequences.keys():
        # Resize each bounding box into size (36, 36) using average pooling
        # Note: only 1220 frames in the first 20 seconds of video are used for illustration
        sequence = np.array(sequences[file_idx][file_idx][:time_len])
        sequence = np.transpose(sequence, (0, 3, 1, 2))
        avg_pool = nn.AdaptiveAvgPool2d((36, 36))
        sequence = avg_pool(torch.from_numpy(sequence).float())

        # Apply bandpath filter for each (pixel, channel) pair sequence
        from scipy.signal import filtfilt
        order = 128
        b = build_bandpass_filter(FPS, order, False)

        _, _, w, h = sequence.size()
        for i in range(w):
            for j in range(h):
                for c in range(3):
                    sequence[:, c, i, j] = torch.from_numpy(filtfilt(b, np.array([1]), sequence[:, c, i, j], axis=0).copy())

        # Make the input seq and target seq to have the same length
        timestamps = timestamps[file_idx]
        instan_hr = instant_hr[file_idx]
        assert len(timestamps) == len(instan_hr)
        total_length = int(min(len(sequence), timestamps[-1]))
        sequence = sequence[: total_length]
        timestamps = [i for i in timestamps if i <= total_length]
        instan_hr = instan_hr[: len(timestamps)]

        # Compute the mean hr seq for every 10 seconds signals from the peak timestamps as groundtruth
        target = []
        for start_point in range(0, total_length - WINDOW_LEN, FPS):
            time_window = [i for i in timestamps if i >= start_point and i < start_point+WINDOW_LEN]
            l_idx = timestamps.index(time_window[0])
            r_idx = timestamps.index(time_window[-1])
            target.append(np.mean(np.array(instan_hr[l_idx: r_idx+1])))

        # apply the sliding window strategy to construct the dataset
        sequence = sequence.tolist()
        seq_windows = [[sequence[i: i + WINDOW_LEN], target[int(i/FPS)]] for i in range(0, total_length - WINDOW_LEN, FPS)]
        all_data.extend(seq_windows)
    return all_data

seq_dir = './dataset/roi_sequence/'
ts_file = './dataset/peak_timestamps_groundtruth_hci_tagging.json'
instant_hr_file = './dataset/instan_hr_groundtruth_hci_tagging.json'
all_data = preprocess_HCI_dataset_for_3dcnn_model(seq_dir, ts_file, instant_hr_file)
print(len(all_data))
# all_data = all_data[:len(all_data)]

All Files: dict_keys(['3396'])
5


### Construct the Training and Testing Dataset
- Construct two types of model input:   
    **Frame input (x_appearance)**: the original content of the resized ROI sequence, which is used in the attention mechanism.  
    **difference input (x_motion)**: the time-domain discrete derivative of the resized ROI sequence, which is the mainsource of pulse information in the 3D-CNN network.  
- Split the whole dataset into two parts: 60% for training, 40% for testing

In [3]:
x_appearance = np.array([x[0] for x in all_data])
x_motion = []
for window in x_appearance:
    t = window[1] - window[0]
    diff = [window[i] - window[i-1] for i in range(1, len(window))]
    x_motion.append(diff)
x_appearance = np.transpose(x_appearance[:, :-1, :, :, :], (0, 2, 1, 3, 4))
x_motion = np.transpose(np.array(x_motion), (0, 2, 1, 3, 4))
# print(x_appearance.shape, x_motion.shape)
x_tensor = torch.from_numpy(np.stack((x_motion, x_appearance), axis=1)).float()
y_tensor = torch.from_numpy(np.array([[x[1]] for x in all_data])).float()
# print(x_tensor.size(), y_tensor.size())
dataset = TensorDataset(x_tensor, y_tensor)

config = {
"n_epoch": 2,
"batch_size": 32
}
train_test_ratio = 0.6
train_num = int(train_test_ratio * (len(dataset)))
train_dataset, test_dataset = random_split(dataset, [train_num, len(dataset)-train_num])

### Model Training
- Split the training dataset into two parts: 80% for training, 20% for validation
- Define the 3D-CNN model, use the MSE loss function and Adam optimizer for training
- In each epoch, train and validate the data in batches

In [4]:
def model_train(config, train_dataset, checkpoint_dir=None):
    val_abs = int(len(train_dataset) * 0.8)
    train_subset, val_subset = random_split(train_dataset, [val_abs, len(train_dataset) - val_abs])

    train_loader = DataLoader(dataset=train_subset, batch_size=int(config["batch_size"]), shuffle=True)
    val_loader = DataLoader(dataset=val_subset, batch_size=int(config["batch_size"]), shuffle=True)

    '''Define the CNN model'''
    model = CNN()
    # print(model.state_dict())
    loss_function = nn.MSELoss() #(reduction='mean')
    optimizer = torch.optim.Adam(model.parameters(), lr=3 * pow(10, -3)) 
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

    '''Model training'''
    model.train()
    for epoch in range(config["n_epoch"]):
        running_loss = []
        for i, data in enumerate(train_loader, 0):
            x_batch, y_batch = data
            # print(x_batch.size(), y_batch.size())

            predictions = model(x_batch)
            # print(predictions.size(), y_batch.size())
            loss = loss_function(predictions, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss.append(loss.item())

        val_loss = []
        mean_error = 0
        model.eval()
        with torch.no_grad():
            predictions_list = []
            targets_list = []
            for i, data in enumerate(val_loader, 0):
                x_batch, y_batch = data
                predictions = model(x_batch)
                # print(predictions.size(), y_batch.size())
                loss = loss_function(predictions, y_batch)
                predictions = predictions.numpy().reshape(-1)
                y_batch = y_batch.numpy().reshape(-1)
                predictions_list.extend(predictions.tolist())
                targets_list.extend(y_batch.tolist())
                mean_error += np.mean(abs(predictions-y_batch))
                val_loss.append(loss.item())
        scheduler.step(sum(val_loss))

        print('epoch: {0} Training loss: {1} Validation loss: {2}'.format(epoch, np.mean(np.array(running_loss)), np.mean(np.array(val_loss))))
    torch.save((model.state_dict(), optimizer.state_dict()), './models/best_model.pt')
    return model

best_trained_model = model_train(config=config, train_dataset=train_dataset)

epoch: 0 Training loss: 3875.448486328125 Validation loss: 3847.62744140625
epoch: 1 Training loss: 3842.102783203125 Validation loss: 3810.165283203125


### Model Testing
- Evaluate the trained model on the testing dataset by mean error, RMSE and correlation coefficient

In [5]:
n_plot = 1 # the No. of the drawed result plot

def test_accuracy(model, test_dataset):
    test_batch_size = 1
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=test_batch_size, shuffle=False)
    model.eval()
    # scaler = StandardScaler()
    with torch.no_grad():
        err_list = []
        rmse_list = []
        cor_list = []
        predictions_list = []
        targets_list = []
        for sequence, targets in test_loader:
#             print(sequence.size())
            predictions = model(sequence)

            # The results are scalers while the code is for vector output
            predictions = predictions.numpy().reshape(-1)
            targets = targets.numpy().reshape(-1)
            errs = abs(predictions - targets)
            err = np.mean(errs)
            rmse = np.sqrt(np.mean(errs**2))
            # print(predictions, targets)

            predictions_list.extend(predictions.tolist())
            targets_list.extend(targets.tolist())
            err_list.append(err)
            rmse_list.append(rmse)
#         draw_picture(predictions_list, targets_list)
        print(predictions_list)
        print(targets_list)
        cor = np.corrcoef(np.array(predictions_list), np.array(targets_list))[0][1]
        cor_list.append(cor)
        print("Best trail test set err: {0} rmse: {1} correlation coefficient: {2} \n".format(np.mean(np.array(err_list)), np.mean(np.array(rmse_list)), np.mean(np.array(cor_list))))
        
test_accuracy(best_trained_model, test_dataset)

[0.49523743987083435, 0.4952322542667389]
[62.1099967956543, 62.288047790527344]
Best trail test set err: 61.70378875732422 rmse: 61.70378875732422 correlation coefficient: -1.0 

